# Data Prep for Crowd Annotation Pipeline

1. Collect raw data 
2. Adjust contrast of images
3. Chop up images into manageable pieces
4. Make into montages
5. Upload to Figure8

Files are named by these scripts such that the code blocks can run back-to-back with minimal input. For this reason, it is recommended that users run through the whole pipeline before processing another set of images.

In [1]:
# import statements
from __future__ import absolute_import

import os

from dcde.pre_annotation.montage_makers import montage_maker, multiple_montage_maker
from dcde.pre_annotation.overlapping_chopper import overlapping_crop_dir
from dcde.pre_annotation.aws_upload import aws_upload, upload
from dcde.pre_annotation.montage_to_csv import csv_maker
from dcde.pre_annotation.fig_eight_upload import fig_eight

In [ ]:
#sometimes raw images are in .tif stacks, not individual .tif files
#optional code block for turning into individual slices

## 2. Adjust contrast of images
description of contrast_adjustment.py

In [ ]:
#contrast_adjustment

## 3. Chop up images into manageable pieces

Each full-size image usually has many cells in it. This makes them difficult to fully annotate! For ease of annotation (and better results), each frame is chopped up into smaller, overlapping frames, ultimately creating a set of movies. 

These smaller movies can be made with overlapping edges, making it easier to stitch annotations together into one large annotated movie (in the post-annotation pipeline). A large overlap will result in redundant annotations.

Even if you want to process the full-sized image, run the chopper with num_segments of 1. The montage makers are written to run on the output of the chopper.

In [4]:
base_direc = "/home/geneva/Desktop/Nb_testing/"
raw_direc = os.path.join(base_direc, "MouseBrain_s7_nuclear")
identifier = "MouseBrain_s7_nuc"
num_x_segments = 5
num_y_segments = 5
overlap_perc = 10

In [5]:
overlapping_crop_dir(raw_direc, identifier, num_x_segments, num_y_segments, overlap_perc)

Current Image Size:  (1024, 1024)
Correct? (y/n): y
Your new images will be  244  pixels by  244  pixels big.
Processing...
Cropped files saved to /home/geneva/Desktop/Nb_testing/MouseBrain_s7_nuclear_chopped_5_5


## 4. Make into montages
multiple_montage_maker is written to run on the output of the chopper, ie the folder where each chopped movie folder is saved. It will make montages of each subfolder according to the variables specified. It will make more than one montage per subfolder if there are enough frames to do so.

The variables used in multiple_montage_maker are saved in a JSON file so they can be reused in post-annotation processing.

In [3]:
montage_len = 15

direc = raw_direc + "_chopped_" + str(num_x_segments) + "_" + str(num_y_segments)
#direc = "/home/geneva/Desktop/Nb_testing/nuclear_test_chopped_4_4"

save_direc = os.path.join(base_direc, identifier + "_montages_" + str(num_x_segments) + "_" + str(num_y_segments))
#save_direc = "/home/geneva/Desktop/Nb_testing/montages"

log_direc = os.path.join(base_direc, "json_logs")

row_length = 5
x_buffer = 5
y_buffer = 5

In [4]:
multiple_montage_maker(montage_len, direc, save_direc, identifier, 
                       num_x_segments, num_y_segments, row_length, x_buffer, y_buffer, log_direc)

Now montaging images from: MouseBrain_s7_nuc_x_00_y_00
You will be able to make 2 montages from this movie.
The last 8 frames will not be used in a montage. 

Now montaging images from: MouseBrain_s7_nuc_x_01_y_00
You will be able to make 2 montages from this movie.
The last 8 frames will not be used in a montage. 

Now montaging images from: MouseBrain_s7_nuc_x_02_y_00
You will be able to make 2 montages from this movie.
The last 8 frames will not be used in a montage. 

Now montaging images from: MouseBrain_s7_nuc_x_03_y_00
You will be able to make 2 montages from this movie.
The last 8 frames will not be used in a montage. 

Now montaging images from: MouseBrain_s7_nuc_x_04_y_00
You will be able to make 2 montages from this movie.
The last 8 frames will not be used in a montage. 

Now montaging images from: MouseBrain_s7_nuc_x_00_y_01
You will be able to make 2 montages from this movie.
The last 8 frames will not be used in a montage. 

Now montaging images from: MouseBrain_s7_nuc_x

## 5. Upload to Figure Eight
Now that the images are processed into montages, they need to be uploaded to an AWS bucket and submitted to Figure Eight. This involves uploading the files to AWS, making a CSV file with the links to the uploaded images, and using that CSV file to create a Figure Eight job.

### Upload files to AWS
aws_upload will look for image files in the specified directory (folder_to_upload, set by default to be wherever the output of multiple_montage_maker was saved) and upload them into a bucket.

For the Van Valen lab, the default bucket is "figure-eight-deepcell" and keys (aws_folder + file names) correspond to the file structure of our data server.

aws_upload returns a list of the urls to which images were uploaded.

In [7]:
#import os

bucket_name = "figure-eight-deepcell" #default
aws_folder = "MouseBrain/set7"
folder_to_upload = save_direc #usually .../montages
#data_to_upload = "/home/geneva/Desktop/Nb_testing/montages/"

uploaded_montages = aws_upload(bucket_name, aws_folder, folder_to_upload)

#os.path.join("https://s3.us-east-2.amazonaws.com", bucket_name, aws_folder)
#print(uploaded_montages)
#from io_utils import get_img_names
#imgs_to_upload = get_img_names(folder_to_upload)
#for index, img in enumerate(imgs_to_upload):
#    print(img)
#    print(os.path.join(folder_to_upload, img))

What is your AWS access key id? ········
What is your AWS secret access key id? ········
Connected to AWS
/home/geneva/Desktop/Files_to_work_through/Data_for_crowd/MouseBrain_s7_nuc_montages_5_5/MouseBrain_s7_nuc_x_0_y_0_montage_0.png  849622 / 849622.0  (100.00%)

/home/geneva/Desktop/Files_to_work_through/Data_for_crowd/MouseBrain_s7_nuc_montages_5_5/MouseBrain_s7_nuc_x_0_y_0_montage_1.png  890794 / 890794.0  (100.00%)

/home/geneva/Desktop/Files_to_work_through/Data_for_crowd/MouseBrain_s7_nuc_montages_5_5/MouseBrain_s7_nuc_x_0_y_1_montage_0.png  967973 / 967973.0  (100.00%)

/home/geneva/Desktop/Files_to_work_through/Data_for_crowd/MouseBrain_s7_nuc_montages_5_5/MouseBrain_s7_nuc_x_0_y_1_montage_1.png  975484 / 975484.0  (100.00%)

/home/geneva/Desktop/Files_to_work_through/Data_for_crowd/MouseBrain_s7_nuc_montages_5_5/MouseBrain_s7_nuc_x_0_y_2_montage_0.png  969750 / 969750.0  (100.00%)

/home/geneva/Desktop/Files_to_work_through/Data_for_crowd/MouseBrain_s7_nuc_montages_5_5/Mouse

### Make CSV file
Figure Eight jobs can be created easily by using a CSV file where each row contains information about one task. For our jobs, each row has the link to the location of one montage, and information about that montage (currently, just the "identifier" specified at the beginning of the pipeline). The CSV file is saved as "identifier".csv in a folder that only holds CSVs. CSV folders are usually in cell-type directories, so identifiers should be able to distinguish between sets, parts, etc.

In [8]:
#identifier = "test"
csv_direc = os.path.join(base_direc, "CSV")

In [9]:
csv_maker(uploaded_montages, identifier, csv_direc)

### Create Figure Eight job
The Figure Eight API allows us to create a new job and upload data to it from this notebook. However, since our jobs don't include required test questions, editing job information such as the title of the job must be done via the website. This section of the notebook uses the API to create a job and upload data to it, then reminds the user to finish editing the job on the website.

Some sample job IDs to copy are provided below.

In [10]:
#job_id_to_copy = 1344258 #Elowitz timelapse RFP pilot
job_id_to_copy = 1346216 #Deepcell MouseBrain 3x5
#job_id_to_copy = 1306431 #Deepcell overlapping Mibi
#job_id_to_copy = 1292179 #Deepcell HEK
#job_id_to_copy =

In [11]:
from fig_eight_upload import fig_eight

fig_eight(csv_direc, identifier, job_id_to_copy)

Figure eight api key? ········
200
New job ID is: 1352262
Added data
Head over to the Figure Eight website to change the name of the job, review it, then contact the success manager so they can launch this job.
